<a href="https://colab.research.google.com/github/olonok69/LLM_Notebooks/blob/main/summarization/Langchain_splitting_GTR-T5_large_Document_classification_no_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq transformers -U --progress-bar off
!pip install sentence-transformers  langchain langchain-community langchain-huggingface trl datasets pypdf  -qqq --progress-bar off
! pip install torch torchvision -qqq --progress-bar off

In [ ]:
! pip install -qU "langchain-chroma>=0.1.2"

In [ ]:
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from langchain.document_loaders import DirectoryLoader, TextLoader

# Loaders
from langchain.schema import Document

# Splitters
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Model


# Embedding Support



# Summarizer we'll use for Map Reduce
from langchain.chains.summarize import load_summarize_chain

# Data Science
import numpy as np
from sklearn.cluster import KMeans


from google.colab import userdata
import logging

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import requests

response = requests.get("https://www.gutenberg.org/cache/epub/64317/pg64317.txt")

In [ ]:
book_complete_text = response.text

In [ ]:
book_complete_text = book_complete_text[5:]

In [ ]:
len(book_complete_text)

In [ ]:
file_path = "/content/drive/MyDrive/data/book.txt"

In [ ]:
with open(file_path, "w", encoding="utf-8") as f:
    f.write(book_complete_text)

In [ ]:

with open(file_path, "r",  encoding="utf-8") as f:
    text = f.read()

In [ ]:
text = text.replace('\t', ' ')

In [ ]:
len(text)

# Text Splitters

- https://python.langchain.com/docs/modules/data_connection/document_transformers/
- https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", "\t"], chunk_size=5000, chunk_overlap=300)



In [ ]:
docs = text_splitter.create_documents([text])
num_documents = len(docs)

print (f"Now our book is split up into {num_documents} documents")

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [ ]:
path_class = "/content/drive/MyDrive/data/classes.csv"

In [ ]:
import os
from typing import List
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import CSVLoader
import logging


def create_vector_db(class_file, db_path, embeddings, metadata_columns):
    """

    Args:
        class_file (_type_): _description_
        chroma_path (_type_): _description_
        embeddings (_type_): _description_
        metadata_columns (_type_): _description_

    Returns:
        _type_: _description_
    """
    loader = CSVLoader(class_file, metadata_columns=metadata_columns)
    documents = loader.load()
    os.makedirs(db_path, exist_ok=True)
    # delete collection if this exists
    db = Chroma.from_documents(
        documents=documents,
        # Chose the embedding you want to use
        embedding=embeddings,
        collection_metadata={"hnsw:space": "cosine"},
    )

    return db


def get_or_create_vector_db(
    class_file: str,
    db_path: str,
    embeddings,
    metadata_columns: List = ["classifier"],
):
    """
    Create Vector Database or read it from persistance location
    """

    logging.info(f"DB VectorStore Path: {db_path}")
    db = create_vector_db(class_file, db_path, embeddings, metadata_columns)

    return db

In [ ]:
db = get_or_create_vector_db(
    class_file=path_class, db_path="/content/drive/MyDrive/chroma/test", embeddings=embeddings
)

In [ ]:
results = {}

for i, doc in enumerate(docs):
    res = db.similarity_search_with_score(doc.page_content, k=5)
    output = []
    for doc_result, score in res:
        output.append({doc_result.metadata.get("classifier"): score})
    results[i] = output



In [ ]:
final_output = {}
for key, value in results.items():
    for item in value:
        key2 = list(item.keys())[0]
        value2 = list(item.values())[0]
        if key2 in final_output:
            final_output[key2] = (final_output[key2] + value2) /2
        else:
            final_output[key2] = value2


In [ ]:
final_output

In [ ]:
import heapq

k_keys_sorted = heapq.nlargest(5, final_output, key=final_output.get)
k_keys_sorted

In [ ]:
sorted(final_output, key=final_output.get, reverse=True)[:5]

In [ ]:
# create output
output = []
for doc
for doc_result, score in res:
    output.append({doc_result.metadata.get("classifier"): score})

In [ ]:
output